In [1]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd

from kasearch import SearchOAS, SetQueries

In [2]:
raw_queries = [
    'QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSS',
    'EVQLQQSGTVLARPGASVKMSCEASGYTFTNYWMHWVKQRPGQGLEWIGAIYPGNSDTSYIQKFKGKAKLTAVTSTTSVYMELSSLTNEDSAVYYCTLYDGYYVFAYWGQGTLVTVSA',
    'QVQLLESGGGLVQPGGSLRLSCAASGFTFSTAAMSWVRQAPGKGLEWVSGISGSGSSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCARELSYLYSGYYFDYWGQGTLVTVSS',
]

In [7]:
QueryDB = SetQueries(raw_queries, n_jobs=2)
QueryDB.queries[0]

Query(aligned_query=array([81, 86, 75,  0, 76, 81, 69, 83, 71, 65,  0, 69, 76, 65, 82, 80, 71,
       65, 83, 86, 75, 76, 83, 67, 75, 65, 83, 71, 89, 84, 70,  0,  0,  0,
        0,  0,  0,  0,  0,  0, 84, 78, 89, 87, 77, 81,  0, 87, 86, 75, 81,
        0, 82,  0, 80,  0, 71,  0, 81,  0,  0, 71,  0, 76, 68,  0, 87, 73,
       71, 65, 73, 89, 80, 71,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       68, 71, 78, 84, 82, 89,  0,  0, 84,  0,  0, 72,  0,  0, 75, 70,  0,
        0, 75,  0,  0,  0, 71, 75, 65, 84, 76, 84, 65,  0, 68,  0,  0,  0,
       75,  0, 83,  0,  0, 83, 83,  0,  0,  0,  0, 84,  0, 65, 89, 77, 81,
       76, 83, 83, 76, 65, 83,  0, 69, 68, 83, 71, 86, 89, 89, 67, 65, 82,
       71, 69, 71, 78,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 89, 65, 87, 70, 65,
       89, 87, 71,  0, 81, 71, 84, 84, 86, 84, 86, 83, 83], dtype=int8), chain='Heavy')

In [4]:
DATABASE_PATH = "../data/test_data/"

In [5]:
search_oas = SearchOAS(database_path=DATABASE_PATH, 
                       allowed_chain='Heavy', 
                       n_jobs=2
                      )

In [6]:
%%timeit -n 1 -r 1
search_oas.search(QueryDB.queries[0], keep_best_n=2)

2min 5s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
%%timeit -n 1 -r 1
search_oas.search(QueryDB.queries[0], keep_best_n=2)

3min 6s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
search_oas.current_best_identities

array([[0.79510413, 0.78571429, 0.83333333],
       [0.79161837, 0.78571429, 0.78409091]])